# WOFOST crop database integration

There is a GitHub repository containing calibtated parameters for a range of varieties of crops. These parameters were used, e.g., in the Hupselbrook test case provided with the SWAP model for potato variety. As these are reusable, it made sense to tap into that database to make creation of valid crop files for pySWAP easier.

In this notebook, we will create a .crp file for potato variety used in the hupselbrook test case.

In [2]:
import pyswap as psp

## Exploring the database

pySWAP contains a simple wrapper around A. Wit's crop database. We can use built in classes to get an overview of available data. Then we can check which crop types are available in the database.

In [12]:
cropdb = psp.WOFOSTCropDB()
cropdb.croptypes

['barley',
 'cassava',
 'chickpea',
 'cotton',
 'cowpea',
 'fababean',
 'groundnut',
 'maize',
 'millet',
 'mungbean',
 'pigeonpea',
 'potato',
 'rapeseed',
 'rice',
 'sorghum',
 'soybean',
 'sugarbeet',
 'sugarcane',
 'sunflower',
 'sweetpotato',
 'tobacco',
 'wheat',
 'seed_onion']


When we call `load_crop_file` on the database object with the name of a specific crop type, we will get a WOFOSTCropFile object. We can then inspect that object to see the file's metadata, parameters in individual sections, available ecotypes and varieties.

In [8]:
cropfile = cropdb.load_crop_file("rice")

In [11]:
cropfile.metadata

{'Creator': 'Allard de Wit',
 'Contributor': None,
 'Contact': 'allard.dewit@wur.nl',
 'Publisher': 'Wageningen Environmental Research',
 'Title': 'Parameter sets for the WOFOST7.2/WOFOST8.0 model for simulating rice',
 'Date': datetime.date(2022, 2, 13),
 'Language': 'EN',
 'Format': 'YAML',
 'Subject': 'WOFOST parameters for rice',
 'Description': 'This file provides parameters for several rice varieties. Mostly calibrated based on observations done at the International Rice Research Institute, but also based on observations from Europe and earlier work.',
 'Identifier': None,
 'Relation': None,
 'Source': None,
 'Type': None,
 'Coverage': {'Region': 'Selected locations in Europe (France, Spain, Greece, Italy, Portugal), tropical regions'},
 'Rights': 'European Union Public License (EUPL)',
 'Keyword': 'rice; Oryza sativa'}

In [13]:
print(cropfile.ecotypes)
print(cropfile.varieties)

['rice_eu', 'rice_irri']
['Rice_501', 'Rice_HYV_IR8', 'Rice_IR64616H_DS', 'Rice_IR64616H_WS', 'Rice_IR64', 'Rice_IR72', 'Rice_IR72_DS', 'Rice_IR72_WS', 'Rice_IR8A']


In [14]:
rice_501 = cropfile.get_variety("Rice_501")

The new object is `CropVariety` object. From here you can view the metadata of the particular variety or obtain just the parameters as a dictionary. Mind that at this stage, all tables are just arrays of numbers, the same way as it is formatted in the .yaml files. pySWAP automatically converts them in tables when they are used to populate the parameters of crop settings classes.

## Using the CropVariety object to populate the crop settings class

The point of having this database integration is to enable populating some of the wofost crop parameters used by swap directly from the files. A lot of the parameters we still have to set manually, namely Preparation, ScheduledIrrigation, Interception, OxygenStress, DroughtStress, etc. 

In [20]:
potato_prep = psp.components.crop.Preparation(
    swprep=0,
    swsow=0,
    swgerm=2,
    tsumemeopt=170.0,
    tbasem=3.0,
    teffmx=18.0,
    hdrygerm=-500.0,
    hwetgerm=-100.0,
    zgerm=-10.0,
    agerm=203.0,
    dvsend=2.0,
    swharv=0,
)

scheduled_irrigation = psp.ScheduledIrrigation(schedule=0)
interception = psp.Interception(swinter=1, cofab=0.25)

potato_chtb = psp.CHTB.create({
    "DVS": [0.0, 1.0, 2.0],
    "CH": [
        1.0,
        40.0,
        50.0,
    ],
})

potato_rdctb = psp.RDCTB.create({"RRD": [0.0, 1.0], "RDENS": [1.0, 0.0]})

potato_ox_stress = psp.components.crop.OxygenStress(
    swoxygen=1,
    swwrtnonox=1,
    aeratecrit=0.5,
    hlim1=-10.0,
    hlim2u=-25.0,
    hlim2l=-25.0,
    swrootradius=2,
    root_radiuso2=0.00015,
)

potato_dr_stress = psp.components.crop.DroughtStress(
    swdrought=1,
    hlim3h=-300.0,
    hlim3l=-500.0,
    hlim4=-10000.0,
    adcrh=0.5,
    adcrl=0.1,
)

Then, we can load the potato_701 variety from the WOFOST database and create the CropDevelopmentSettings object like so:

In [21]:
db = psp.WOFOSTCropDB()
potato = db.load_crop_file("potato")
potato_params = potato.get_variety("Potato_701")

potato_cropdev_settings = psp.components.crop.CropDevelopmentSettingsWOFOST(
    wofost_variety=potato_params,
    swcf=2,
    dvs_ch=potato_chtb,
    albedo=0.19,
    laiem=0.0589,
    ssa=0.0,
    kdif=1.0,
    rsc=207.0,
    rsw=0.0,
    kdir=0.75,
    eff=0.45,
    swrd=2,
    rdc=50.0,
    swdmi2rd=1,
    rdctb=potato_rdctb,
)

# By calling the `update_from_wofost method`, the CropDevelopmentSettingsWOFOST object
# will be updated with the values from the WOFOST crop file and the model will be evaluated,
# to make sure that some of the required settings are not missing.
potato_cropdev_settings.update_from_wofost()

CropDevelopmentSettingsWOFOST(name='cropdevelopmentsettingswofost', wofost_variety=None, swcf=2, dvs_cf=None, dvs_ch=   DVS    CH
0  0.0   1.0
1  1.0  40.0
2  2.0  50.0, albedo=0.19, rsc=207.0, rsw=0.0, tsum1=150.0, tsum2=1550.0, tbase=2.0, kdif=1.0, kdir=0.75, swrd=2, rdtb=None, rdi=10.0, rri=1.2, rdc=50.0, swdmi2rd=1, rlwtb=None, wrtmax=None, swrdc=0, rdctb=   RRD  RDENS
0  0.0    1.0
1  1.0    0.0, idsl=0, dtsmtb=    TAV  DTSM
0   0.0   0.0
1   2.0   0.0
2  13.0  11.0
3  30.0  28.0, dlo=None, dlc=None, vernsat=70.0, vernbase=14.0, verndvs=0.3, verntb=None, tdwi=75.0, laiem=0.0589, rgrlai=0.012, spa=0.0, ssa=0.0, span=37.0, slatb=   DVS     SLA
0  0.0  0.0030
1  1.1  0.0030
2  2.0  0.0015, eff=0.45, amaxtb=    DVS  AMAX
0  0.00  30.0
1  1.57  30.0
2  2.00   0.0, tmpftb=   TAVD  TMPF
0   0.0  0.01
1   3.0  0.01
2  10.0  0.75
3  15.0  1.00
4  20.0  1.00
5  26.0  0.75
6  33.0  0.01, tmnftb=   TMNR  TMNF
0   0.0   0.0
1   3.0   1.0, cvo=0.85, cvl=0.72, cvr=0.72, cvs=0.69, q10=2.0, rml=0.

Further we proceed with creation of the `CropFile` object, which goes directly to the `Model`. You can observe the content of the crop file by calling `CropFile.crp` property.

In [22]:
crppotato = psp.components.crop.CropFile(
    name="potatod",
    prep=potato_prep,
    cropdev_settings=potato_cropdev_settings,
    oxygenstress=potato_ox_stress,
    droughtstress=potato_dr_stress,
    interception=interception,
    scheduledirrigation=scheduled_irrigation,
)

In [24]:
print(crppotato.crp)

SWPREP = 0
SWSOW = 0
SWGERM = 2
SWHARV = 0
DVSEND = 2.0
TSUMEMEOPT = 170.0
TBASEM = 3.0
TEFFMX = 18.0
HDRYGERM = -500.0
HWETGERM = -100.0
ZGERM = -10.0
AGERM = 203.0
SWCF = 2
 DVS   CH
 0.0  1.0
 1.0 40.0
 2.0 50.0

ALBEDO = 0.19
RSC = 207.0
RSW = 0.0
TSUMEA = 150.0
TSUMAM = 1550.0
TBASE = 2.0
KDIF = 1.0
KDIR = 0.75
SWRD = 2
RDI = 10.0
RRI = 1.2
RDC = 50.0
SWDMI2RD = 1
SWRDC = 0
RDCTB = 
0.0 1.0
1.0 0.0

IDSL = 0
DTSMTB = 
 0.0  0.0
 2.0  0.0
13.0 11.0
30.0 28.0

VERNSAT = 70.0
VERNBASE = 14.0
VERNDVS = 0.3
TDWI = 75.0
LAIEM = 0.0589
RGRLAI = 0.012
SPA = 0.0
SSA = 0.0
SPAN = 37.0
SLATB = 
0.0 0.0030
1.1 0.0030
2.0 0.0015

EFF = 0.45
AMAXTB = 
0.00 30.0
1.57 30.0
2.00  0.0

TMPFTB = 
 0.0 0.01
 3.0 0.01
10.0 0.75
15.0 1.00
20.0 1.00
26.0 0.75
33.0 0.01

TMNFTB = 
0.0 0.0
3.0 1.0

CVO = 0.85
CVL = 0.72
CVR = 0.72
CVS = 0.69
Q10 = 2.0
RML = 0.03
RMO = 0.0045
RMR = 0.01
RMS = 0.015
RFSETB = 
0.0 1.0
2.0 1.0

FRTB = 
0.00 0.2
1.00 0.2
1.36 0.0
2.00 0.0

FLTB = 
0.00 0.8
1.00 0.8
1.27 0.0
1.

The WOFOST database integration allows you draw calibrated settings from the database into pySWAP. If you wish to change some of the settings, you should do it by updating the base pySWAP classes (e.g., CropDevalopmentSettings), like so:

In [30]:
potato_cropdev_settings_tsum1_up = potato_cropdev_settings.model_copy(
    update={"tsum1": 900}
)

In [32]:
crppotato_tsum1_up = crppotato.model_copy(
    update={"cropdev_settings": potato_cropdev_settings_tsum1_up}
)

In [33]:
print(crppotato_tsum1_up.crp)

SWPREP = 0
SWSOW = 0
SWGERM = 2
SWHARV = 0
DVSEND = 2.0
TSUMEMEOPT = 170.0
TBASEM = 3.0
TEFFMX = 18.0
HDRYGERM = -500.0
HWETGERM = -100.0
ZGERM = -10.0
AGERM = 203.0
SWCF = 2
 DVS   CH
 0.0  1.0
 1.0 40.0
 2.0 50.0

ALBEDO = 0.19
RSC = 207.0
RSW = 0.0
TSUMEA = 900.0
TSUMAM = 1550.0
TBASE = 2.0
KDIF = 1.0
KDIR = 0.75
SWRD = 2
RDI = 10.0
RRI = 1.2
RDC = 50.0
SWDMI2RD = 1
SWRDC = 0
RDCTB = 
0.0 1.0
1.0 0.0

IDSL = 0
DTSMTB = 
 0.0  0.0
 2.0  0.0
13.0 11.0
30.0 28.0

VERNSAT = 70.0
VERNBASE = 14.0
VERNDVS = 0.3
TDWI = 75.0
LAIEM = 0.0589
RGRLAI = 0.012
SPA = 0.0
SSA = 0.0
SPAN = 37.0
SLATB = 
0.0 0.0030
1.1 0.0030
2.0 0.0015

EFF = 0.45
AMAXTB = 
0.00 30.0
1.57 30.0
2.00  0.0

TMPFTB = 
 0.0 0.01
 3.0 0.01
10.0 0.75
15.0 1.00
20.0 1.00
26.0 0.75
33.0 0.01

TMNFTB = 
0.0 0.0
3.0 1.0

CVO = 0.85
CVL = 0.72
CVR = 0.72
CVS = 0.69
Q10 = 2.0
RML = 0.03
RMO = 0.0045
RMR = 0.01
RMS = 0.015
RFSETB = 
0.0 1.0
2.0 1.0

FRTB = 
0.00 0.2
1.00 0.2
1.36 0.0
2.00 0.0

FLTB = 
0.00 0.8
1.00 0.8
1.27 0.0
1.